In [9]:
import yfinance as yf
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)

test = yf.Ticker("MSFT")

msft_market_cap = test.info['marketCap']
msft_current_assets = test.quarterly_balance_sheet.loc['Total Assets'][0]
msft_total_liabilities = test.quarterly_balance_sheet.loc['Current Liabilities'][0]
msft_shares_outstanding = test.info['sharesOutstanding']
msft_hist = test.history(period="5y", interval="3mo")
msft_hist['marketCap'] = msft_shares_outstanding * msft_hist['Close']
msft_free_cash_flow_recent_quarter = test.cash_flow.loc['Free Cash Flow'][0]
msft_dividends = test.history(period="5y")["Dividends"][test.history(period="5y")["Dividends"] > 0].to_list()
msft_dividends_paid = (test.history(period="5y")["Dividends"] > 0).sum()
msft_total_debt = test.quarterly_balance_sheet.loc['Total Debt'][0]
msft_cash_and_equivalents_and_short_term_investments = test.quarterly_balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'][0]

msft_fcf = pd.DataFrame(test.cash_flow.loc['Free Cash Flow'])
msft_fcf.index = msft_fcf.index + pd.Timedelta(days=1)
msft_fcf.index = [str(i)[:10] for i in msft_fcf.index]
msft_hist.index = [str(i)[:10] for i in msft_hist['marketCap'].index]
df1 = pd.DataFrame(msft_hist['marketCap'])

result = pd.merge(msft_fcf, df1, left_index=True, right_index=True)

False

In [1]:
from screener import Screener

screener = Screener()


## 1. Market Cap at or below Net-Current-Asset-Value

In [11]:
market_cap = msft_market_cap
current_assets = msft_current_assets
total_liabilities = msft_total_liabilities

market_cap #<= screener.calculate_ncav(current_assets, total_liabilities)

3009100644352

## 2. Average 5Y annual Free Cash Flow yield at 10% or more

In [12]:
# Example usage:
fcff_values = [100000, 120000, 150000, 130000, 110000]
market_cap_values = [800000, 900000, 1000000, 950000, 850000]

screener.calculate_average_earnings_yield(fcff_values, market_cap_values)

Yields >=10% [12.5, 13.333333333333334, 15.0, 13.684210526315791, 12.941176470588237]


13.0

## 2.1 Average 4Y annual Free Cash Flow yield at 10% or more

In [15]:
fcff_values = result['Free Cash Flow']
market_cap_values = result['marketCap']

f = screener.calculate_average_earnings_yield(fcff_values, market_cap_values)

type(f)

Yields >=10% []


int

## 3. Positive TTM Free-cashflow

In [13]:
values = msft_free_cash_flow_recent_quarter
ttm = values

ttm 

59475000000.0

## 4. Zero or negative net-debt

In [6]:
total_debt = msft_total_debt
cash_and_equivalents = msft_cash_and_equivalents_and_short_term_investments
screener.calculate_net_debt(total_debt, cash_and_equivalents) 

-58956000000.0

## 5. Some 5Y dividends or buybacks present

## Test `Screener.execute(args)`

In [7]:
msft = {"Ticker":"MSFT", "Market Caps":[8000, 9000, 10000, 9500, 750], "Current Assets":5000,"Total Liabilities":30, "Free Cash Flows":[100000, 120000, 150000, 130000, 110000], "Total Debt":-400, "Cash":320, "Dividends":[0,0,0,0,0.5,0.5]}

scr = Screener()
scr.execute(msft["Ticker"], msft['Market Caps'], msft['Current Assets'], msft['Total Liabilities'], msft['Free Cash Flows'], msft['Total Debt'], msft['Cash'], msft["Dividends"])
scr.newly_added


Yields >=10% [1250.0, 1333.3333333333335, 1500.0, 1368.421052631579, 14666.666666666666]


{'MSFT': {'Market Cap at or below NCAV': True,
  'Average 5Y annual Free Cash Flow yield at 10%': True,
  'TTM is Positive': True,
  'Net-Debt': -720,
  'Has paid dividends or buybacks': True}}

## Calculate Payback Rating